In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [12]:
def get_jobs(keyword, num_jobs, verbose,sleep_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/Nantha/Documents/Glassdoor-Job-Salary-Prediction/glassdoor-scrapper/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.generalKeyword="'+keyword+'"&sc.locationSeoString=san+jose&locId=1147436&locT=C'
#     url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(sleep_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [13]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 15, False,12)
df

Progress: 0/15
Progress: 1/15
Progress: 2/15
Progress: 3/15
Progress: 4/15
Progress: 5/15
Progress: 6/15
Progress: 7/15
Progress: 8/15
Progress: 9/15
Progress: 10/15
Progress: 11/15
Progress: 12/15
Progress: 13/15
Progress: 14/15
Progress: 15/15


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Principal Data Scientist (Python/R),$212K-$332K (Glassdoor Est.),Roche Diagnostics has built a new strategic ar...,4.1,Roche\n4.1,"Santa Clara, CA","Basel, Switzerland",10000+ employees,1896,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹500+ billion (INR),"Novartis, AstraZeneca, Siemens Healthineers"
1,Data Scientist 1,$212K-$332K (Glassdoor Est.),We are looking to hire an analytics & insights...,3.6,eBay\n3.6,"San Jose, CA","San Jose, CA",10000+ employees,1995,Company - Public,Internet,Information Technology,₹500+ billion (INR),"Amazon, Apple"
2,"Data Scientist, Finance",$212K-$332K (Glassdoor Est.),Zoom is an award-winning workplace. We have be...,4.7,Zoom Video Communications\n4.7,"San Jose, CA","San Jose, CA",1001 to 5000 employees,2011,Company - Public,IT Services,Information Technology,Unknown / Non-Applicable,-1
3,Data Scientist — Risk,$212K-$332K (Glassdoor Est.),Data Scientist — Risk\nMountain ViewR&DExperie...,4.1,TikTok\n4.1,"Mountain View, CA","Los Angeles, CA",1001 to 5000 employees,2016,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1
4,Sr. Data Scientist II,$212K-$332K (Glassdoor Est.),Shutterfly’s purpose is to help share life’s j...,3.1,Shutterfly\n3.1,"Redwood City, CA","Redwood City, CA",1001 to 5000 employees,1999,Company - Private,Internet,Information Technology,₹100 to ₹500 billion (INR),"Snapfish, Vistaprint, Minted"
5,Research Data Scientist,$212K-$332K (Glassdoor Est.),About the team\n\nOur Data Science team is an ...,3.9,Roku\n3.9,"San Jose, CA","San Jose, CA",1001 to 5000 employees,2002,Company - Public,IT Services,Information Technology,₹50 to ₹100 billion (INR),-1
6,Data Scientist,$212K-$332K (Glassdoor Est.),"At Waymo, we're focused on bringing self-drivi...",3.1,Waymo\n3.1,"Mountain View, CA","Mountain View, CA",Unknown,2009,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
7,"Data Scientist, Operations Data Science",$212K-$332K (Glassdoor Est.),Minimum qualifications:\nMaster's degree in Op...,4.4,Google\n4.4,"Mountain View, CA","Mountain View, CA",10000+ employees,1998,Company - Public,Internet,Information Technology,₹500+ billion (INR),"Microsoft, Apple, Facebook"
8,Data Scientist,$212K-$332K (Glassdoor Est.),RMS is the pioneer in the development and appl...,3.5,Risk Management Solutions (RMS)\n3.5,"Newark, CA","Newark, CA",1001 to 5000 employees,1989,Company - Private,Enterprise Software & Network Solutions,Information Technology,₹10 to ₹50 billion (INR),"AIR Worldwide, EQECAT, Verisk Analytics"
9,Data Scientist #5967,$212K-$332K (Glassdoor Est.),Company Overview\n\nFanatics is the global lea...,3.3,Fanatics\n3.3,"San Mateo, CA","Jacksonville, FL",5001 to 10000 employees,1996,Company - Private,Sporting Goods Shops,Retail,₹100 to ₹500 billion (INR),"Lids, Amazon, Under Armour"
